In [1]:
using DataFrames
using CSV
using Plots
using LaTeXStrings
using StatsPlots
using Statistics
using HypothesisTests

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1278
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278
┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1278
┌ Info: Precompiling HypothesisTests [09f84164-cd44-5f33-b23f-e6b0d136a0d5]
└ @ Base loading.jl:1278


In [2]:
plotly()
theme(:default)

In [3]:
function plot_C(num_idx)
    num_zs = []
    hps = []
    n_zs = 0
    for C in ["0.0001", "0.001", "0.01", "0.1", "1.0", "10.0"], index in 1:num_idx 
        open("../insights/mall/mall_slice_$(C)_0.1_0.001_$index.csv", "r") do io
            for (num, line) in enumerate(eachline(io))
                if num == 2
                    tmps = split(strip(line), '\t')
                    n_zs = parse(Float64, tmps[4])
                end 
            end 
            push!(hps, C)
            push!(num_zs, n_zs)
        end 
    end
        
    C_df = DataFrame(
        C = hps,
        num_zs = num_zs
    )

    xlb = L"$C$" 
    p = @df C_df boxplot(:C, :num_zs,  
                         marker=(0.5, :black, stroke(2)),
                         bar_width=.25,
                         lw=2,
                         size=(360, 450),
#                            alpha=.55,
                         color="#ffa31a",
#                            linecolor=:skyblue,
                         framestyle=:semi,
                         tickfontsize=14,
                         legend=:best,
                         legendfontsize=14,
                         xlabel=xlb,
                         ci=95,
                         xguidefontsize=18,
                         label="NO. of nodes")

    return p
end


plot_C (generic function with 1 method)

In [4]:
p = plot_C(35)


<!DOCTYPE html>
 
 
 Plots.jl

┌ Warning: Framestyle :semi is not supported by Plotly and PlotlyJS. :box was cosen instead.
└ @ Plots /home/kra/.julia/packages/Plots/oZheM/src/backends/plotly.jl:10


In [5]:
savefig(p, "../plots/mall/C_analysis.pdf")

# C

In [3]:
num_zs = []
hps = []
llkhds = []
for C in ["0.0001", "0.1", "1.0", "10.0", "10000.0"], index in 1:50 
    n_zs = 0
    open("../insights/mall/mall_slice_1.0_$(C)_0.01_0.1_$index.csv", "r") do io
        for (num, line) in enumerate(eachline(io))
            if num == 2
                tmps = split(strip(line), '\t')
                n_zs = parse(Int, tmps[5])
#                 C == "0.0001" && println(n_zs)
                push!(num_zs, n_zs)
                push!(llkhds, parse(Float64, tmps[6]))
            end 
        end 
        push!(hps, C)
    end 
end

C_df = DataFrame(
    C = hps,
    num_nodes = num_zs,
    llkhds = llkhds
);

In [4]:
gd = groupby(C_df, :C);

# combine(C_df, , :num_nodes => mean)
C_df

250×3 DataFrame
 Row │ C        num_nodes  llkhds     
     │ Any      Any        Any        
─────┼────────────────────────────────
   1 │ 0.0001   7          -2.04886e5
   2 │ 0.0001   9          -2.02802e5
   3 │ 0.0001   7          -2.04939e5
   4 │ 0.0001   4          -9.07335e5
   5 │ 0.0001   7          -204905.0
   6 │ 0.0001   10         -1.70216e5
   7 │ 0.0001   9          -1.88078e5
   8 │ 0.0001   10         -1.07488e5
   9 │ 0.0001   4          -3.69092e5
  10 │ 0.0001   4          -3.7002e5
  11 │ 0.0001   7          -2.04844e5
  ⋮  │    ⋮         ⋮          ⋮
 241 │ 10000.0  13         -332876.0
 242 │ 10000.0  15         -5.6546e5
 243 │ 10000.0  29         -224977.0
 244 │ 10000.0  9          -5.66396e5
 245 │ 10000.0  9          -3.30827e5
 246 │ 10000.0  13         -7.14679e5
 247 │ 10000.0  22         -2.83091e5
 248 │ 10000.0  10         -1.00948e6
 249 │ 10000.0  10         -4.25875e5
 250 │ 10000.0  7          -7.90189e5
                      229 rows omitted

In [5]:
combine(gd, :num_nodes => mean, 
            :num_nodes => std, 
            :llkhds => mean,
            :llkhds => std)

5×5 DataFrame
 Row │ C        num_nodes_mean  num_nodes_std  llkhds_mean      llkhds_std 
     │ Any      Float64         Float64        Float64          Float64    
─────┼─────────────────────────────────────────────────────────────────────
   1 │ 0.0001             8.82        3.47991  -271120.0         2.51433e5
   2 │ 0.1                8.94        3.371    -264908.0         2.35723e5
   3 │ 1.0                8.1         3.19598       -2.77506e5   2.55485e5
   4 │ 10.0               8.78        4.99097       -2.5735e5    2.23548e5
   5 │ 10000.0           11.78        5.31187       -1.11525e6   1.50767e6

In [6]:
for g in gd
    ci = confint(OneSampleTTest(g[:, :num_nodes] |> Vector{Real}))
    println(g[1, 1]," ==> ci is $ci")
end 

0.0001 ==> ci is (7.83101960957621, 9.808980390423791)
0.1 ==> ci is (7.981971031436367, 9.898028968563633)
1.0 ==> ci is (7.191712642530951, 9.008287357469047)
10.0 ==> ci is (7.361581610784162, 10.198418389215837)
10000.0 ==> ci is (10.270384440413414, 13.289615559586585)


# Discount

In [7]:
num_zs = []
hps = []
llkhds = []

for disc in ["0.0001", "0.1", "1.0", "10.0", "10000.0"], index in 1:50 
    n_zs = 0
    open("../insights/mall/mall_slice_1.0_1.0_$(disc)_0.1_$index.csv", "r") do io
        for (num, line) in enumerate(eachline(io))
            if num == 2
                tmps = split(strip(line), '\t')
                n_zs = parse(Int, tmps[5])
#                 C == "0.0001" && println(n_zs)
                push!(num_zs, n_zs)
                push!(llkhds, parse(Float64, tmps[6]))
            end 
        end 
        push!(hps, disc)
    end 
end

disc_df = DataFrame(
    disc = hps,
    num_nodes = num_zs,
    llkhds = llkhds
);

In [8]:
disc_gd = groupby(disc_df, :disc);

combine(disc_gd, :num_nodes => mean, 
                :num_nodes => std, 
                :llkhds => mean,
                :llkhds => std)

5×5 DataFrame
 Row │ disc     num_nodes_mean  num_nodes_std  llkhds_mean  llkhds_std 
     │ Any      Float64         Float64        Float64      Float64    
─────┼─────────────────────────────────────────────────────────────────
   1 │ 0.0001             7.54        2.65891  -3.05285e5   3.00646e5
   2 │ 0.1                7.88        2.43813  -2.18329e5   1.22056e5
   3 │ 1.0                9.96        3.47563  -2.65807e5   2.45304e5
   4 │ 10.0               9.68        3.67807  -7.78801e5   1.88897e6
   5 │ 10000.0            8.1         3.41216  -4.38107e11  1.42587e12

In [9]:
for g in disc_gd
    ci = confint(OneSampleTTest(g[:, :num_nodes] |> Vector{Real}))
    println(g[1, 1]," ==> ci is $ci")
end 

0.0001 ==> ci is (6.7843464847117145, 8.295653515288285)
0.1 ==> ci is (7.1870904272152, 8.5729095727848)
1.0 ==> ci is (8.972237042455113, 10.947762957544889)
10.0 ==> ci is (8.634705449379231, 10.725294550620768)
10000.0 ==> ci is (7.130273971466804, 9.069726028533196)


In [102]:
xlb = L"$\tau$" 

p = @df disc_df boxplot(:disc, :num_nodes,  
                         marker=(0.8, :black, stroke(1)),
                         bar_width=.45,
                         lw=2,
#                            size=(600, 430),
#                            alpha=.55,
                         color="#ffa31a",
#                            linecolor=:skyblue,
                         framestyle=:semi,
                         tickfontsize=14,
                         legend=:best,
                         legendfontsize=14,
                         xlabel=xlb,
                         ci=95,
                         xguidefontsize=18,
                         label="NO. of nodes")


<!DOCTYPE html>
 
 
 Plots.jl

┌ Warning: Framestyle :semi is not supported by Plotly and PlotlyJS. :box was cosen instead.
└ @ Plots /home/kra/.julia/packages/Plots/oZheM/src/backends/plotly.jl:10


In [103]:
savefig(p, "../plots/mall/tau_analysis.pdf")

# nu0

In [10]:
num_zs = []
hps = []
llkhds = []
for ss in ["0.0001", "0.1", "1.0", "10.0", "10000.0"], index in 1:50
    n_zs = 0
    open("../insights/mall/mall_slice_1.0_1.0_0.01_$(ss)_$index.csv", "r") do io
        for (num, line) in enumerate(eachline(io))
            if num == 2
                tmps = split(strip(line), '\t')
                n_zs = parse(Int, tmps[5])
#                 C == "0.0001" && println(n_zs)
                push!(num_zs, n_zs)
                push!(llkhds, parse(Float64, tmps[6]))
            end 
        end 
        push!(hps, ss)
    end 
end

ss_df = DataFrame(
    ss = hps,
    num_nodes = num_zs,
    llkhds = llkhds
);

In [11]:
ss_gd = groupby(ss_df, :ss);

ss_stats = combine(ss_gd, :num_nodes => mean, 
                            :num_nodes => std, 
                            :llkhds => mean,
                            :llkhds => std)

5×5 DataFrame
 Row │ ss       num_nodes_mean  num_nodes_std  llkhds_mean      llkhds_std 
     │ Any      Float64         Float64        Float64          Float64    
─────┼─────────────────────────────────────────────────────────────────────
   1 │ 0.0001             7.44        2.94272  -321123.0         2.74432e5
   2 │ 0.1                8.1         3.19598       -2.77506e5   2.55485e5
   3 │ 1.0                8.08        2.87736       -2.8035e5    2.599e5
   4 │ 10.0               7.74        2.95414  -312148.0         2.88817e5
   5 │ 10000.0            7.92        2.78363       -2.90439e5   2.2129e5

In [12]:
for g in ss_gd
    ci = confint(OneSampleTTest(g[:, :num_nodes] |> Vector{Real}))
    println(g[1, 1]," ==> ci is $ci")
end 

0.0001 ==> ci is (6.603688671795352, 8.276311328204649)
0.1 ==> ci is (7.191712642530951, 9.008287357469047)
1.0 ==> ci is (7.262264170286667, 8.897735829713334)
10.0 ==> ci is (6.900442911224934, 8.579557088775067)
10000.0 ==> ci is (7.128902360277699, 8.7110976397223)


In [16]:
xlb = L"$\sigma^2$" 

p = @df ss_df boxplot(:ss, :num_nodes,  
                         marker=(0.8, :black, stroke(1)),
                         bar_width=.45,
                         lw=2,
#                            size=(600, 430),
#                            alpha=.55,
                         color="#ffa31a",
#                            linecolor=:skyblue,
                         framestyle=:semi,
                         tickfontsize=14,
                         legend=:best,
                         legendfontsize=14,
                         xlabel=xlb,
                         ci=95,
                         xguidefontsize=18,
                         label="NO. of nodes")

<!DOCTYPE html>
 
 
 Plots.jl

┌ Warning: Framestyle :semi is not supported by Plotly and PlotlyJS. :box was cosen instead.
└ @ Plots /home/kra/.julia/packages/Plots/SIQaY/src/backends/plotly.jl:10


In [18]:

p = @df ss_stats scatter(:ss, :num_nodes_mean,  
     bar_width=.45,
     lw=2,
     color="#ffa31a",
     framestyle=:semi,
     tickfontsize=14,
     legend=:best,
     legendfontsize=14,
     xlabel=xlb,
     ci=95,
     xguidefontsize=18,
     label="NO. of nodes")

<!DOCTYPE html>
 
 
 Plots.jl

┌ Warning: Framestyle :semi is not supported by Plotly and PlotlyJS. :box was cosen instead.
└ @ Plots /home/kra/.julia/packages/Plots/SIQaY/src/backends/plotly.jl:10


# epsilon0

In [14]:
num_zs = []
hps = []
llkhds = []
for es in ["0.0001", "0.1", "1.0", "10.0", "10000.0"], index in 1:50
    n_zs = 0
    open("../insights/mall/mall_slice_$(es)_1.0_0.01_0.1_$index.csv", "r") do io
        for (num, line) in enumerate(eachline(io))
            if num == 2
                tmps = split(strip(line), '\t')
                n_zs = parse(Int, tmps[5])
#                 C == "0.0001" && println(n_zs)
                push!(num_zs, n_zs)
                push!(llkhds, parse(Float64, tmps[6]))
            end 
        end 
        push!(hps, es)
    end 
end

es_df = DataFrame(
    es = hps,
    num_nodes = num_zs,
    llkhds = llkhds
);

In [15]:
es_gd = groupby(es_df, :es);

es_stats = combine(es_gd, :num_nodes => mean, 
                            :num_nodes => std, 
                            :llkhds => mean,
                            :llkhds => std)

5×5 DataFrame
 Row │ es       num_nodes_mean  num_nodes_std  llkhds_mean  llkhds_std 
     │ Any      Float64         Float64        Float64      Float64    
─────┼─────────────────────────────────────────────────────────────────
   1 │ 0.0001             8.04        2.59481   -2.63046e5   2.24786e5
   2 │ 0.1                8.38        2.76929   -2.51244e5   1.94758e5
   3 │ 1.0                8.1         3.19598   -2.77506e5   2.55485e5
   4 │ 10.0               9.9         6.37037   -3.1066e5    2.84465e5
   5 │ 10000.0           16.9         6.21141   -6.52167e5   9.89382e5